In [1]:
import numpy as np

In [2]:
# 使用贝叶斯算法进行文本分类，对网站留言评论进行分类：侮辱类和非侮辱类
def loadDataSet():
    # 实验样本、文档list
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]  # 样本类别，1表示是侮辱性的、0表示非侮辱性的留言
    return postingList, classVec


def createVocabList(dataSet):
    vocabSet = set([])  # 创建空集合，set也会去除重复的元素
    for document in dataSet:
        vocabSet = vocabSet | set(document) # |为按位或运算符，取两个集合的并集
    return list(vocabSet)  # set类型是无序的，所以每次运行返回的顺序不同


def setOfWords2Vec(vocabList, inputSet):
    # 初始化一个长度相同，全为0的list
    returnVec = [0]*len(vocabList)
    # 遍历给定样本的词汇
    for word in inputSet:
        if word in vocabList:  # 判断并对出现的词汇设置8为1
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

post_list, classvec = loadDataSet()
not_repeatlist = createVocabList(post_list)
print(not_repeatlist)
out = setOfWords2Vec(not_repeatlist, post_list[3])
print(out)

['so', 'I', 'steak', 'how', 'help', 'mr', 'maybe', 'food', 'worthless', 'flea', 'stop', 'cute', 'ate', 'is', 'has', 'licks', 'park', 'please', 'quit', 'problems', 'not', 'buying', 'my', 'dog', 'dalmation', 'love', 'to', 'posting', 'him', 'stupid', 'take', 'garbage']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]


In [3]:
# 训练、分类
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)  # P(c)
    # 运用拉普拉斯平滑，避免了其中一个概率值为0导致整个式子为0的情况
    # 拉普拉斯平滑原理，分子每一项加上一个大于0的数，eg：+1，分母的所有项也+1(因为是伯努利分布，只有两项，所以分母+2)
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)      
    p0Denom = 2.0; p1Denom = 2.0                        
    for i in range(numTrainDocs):
        # 当为侮辱性留言时
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]  # 叠加词汇出现的次数
            p1Denom += sum(trainMatrix[i])  # 总数，分母，用于求频率即概率
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # 使用log避免下溢出的情况(大量很小的数相乘最终python会四舍五入输出为0)
    # 两个条件概率
    p1Vect = np.log(p1Num/p1Denom)         
    p0Vect = np.log(p0Num/p0Denom)          
    return p0Vect,p1Vect,pAbusive

为了防止分子的每一个相乘的子项不为0，则分子的条件概率变为：$$P_{\lambda }(X^{j}=a_{jl}|Y=c_{k})=\frac{\sum_{N}^{i=1}I(x_{i}^{j}=a_{jl},y_{i}=c_{k})+\lambda }{\sum_{N}^{i=1}I(y_{i}=c_{k})+S_{j}\lambda }$$I为指示函数，$S_{j}$代表种类的个数，具体参考《概率论与数理统计》浙大第四版

In [4]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # 因为分子运用了log运算，所以sum和+是合理的
    # 其实p1和p0不是真正的概率值，也不需要求出真实概率值，可以判断大小即可，你会发现下面两个式子中连分母(P(w))都没有用到，因为分母都一样比较分子即可
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)    
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
    
post_list, classvec = loadDataSet()
not_repeatlist = createVocabList(post_list)
traindata = []
for postindoc in post_list:
    traindata.append(setOfWords2Vec(not_repeatlist, postindoc))
p1, p2, p3 = trainNB0(traindata, classvec)
test0 = ['love', 'my', 'dalmation']
test2num = setOfWords2Vec(not_repeatlist, test0)
out = classifyNB(test2num, p1, p2, p3)
print("该留言类别是：", out)

该留言类别是： 0


In [6]:
# 没什么~就是一个测试样本的封装函数
def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(np.array(trainMat), np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


In [13]:
# # 当词汇在文本中出现不止一次时，使用朴素贝叶斯词袋模型
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            # 词袋模型对出现的词汇进行次数累加
            returnVec[vocabList.index(word)] += 1
    return returnVec


# 电子邮件垃圾过滤
###################################################
def textParse(bigString):  # 从文本文档构建词列表
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]


def spamTest():
    docList = [] # 文本列表
    classList = [] # 类别列表
    fullText = []
    # 加载读取data
    for i in range(1, 26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    # 去除重复的列表
    vocabList = createVocabList(docList)  
    trainingSet = list(range(50))
    testSet = []  # create test set
    # 随机选取10个作为测试样本
    for i in range(10):
        randIndex = int(np.random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del trainingSet[randIndex]  # 并将测试样本从数据集中删除
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:  
        # 转换成数字向量
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    # 训练
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:  # 测试
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        # 利用训练集求出的三个概率值测试test样本
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
            print("classification error", docList[docIndex])
    print('the error rate is: ', float(errorCount) / len(testSet))
    # return vocabList,fullText

spamTest()

the error rate is:  0.0


E:\Anaconda3\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [15]:
spamTest()   # 测试样本是随机选出的，错误率不同也合理

classification error ['yay', 'you', 'both', 'doing', 'fine', 'working', 'mba', 'design', 'strategy', 'cca', 'top', 'art', 'school', 'new', 'program', 'focusing', 'more', 'right', 'brained', 'creative', 'and', 'strategic', 'approach', 'management', 'the', 'way', 'done', 'today']
the error rate is:  0.1


E:\Anaconda3\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
